In [101]:
import numpy as np
from scipy.sparse.linalg import eigsh
from scipy import sparse
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [155]:
N_points = 100
up_bound = 1
dn_bound = -1

k = 1000
M = 1
h_bar = 1
omega = np.sqrt(k/M)

del_s = (up_bound - dn_bound)/(N_points - 1)

x_grid,y_grid = np.meshgrid(
    np.linspace(dn_bound,up_bound,N_points,dtype=float),
    np.linspace(dn_bound,up_bound,N_points,dtype=float)
    )

D = sparse.spdiags(
    np.array([np.ones(N_points), -2*np.ones(N_points), np.ones(N_points)]),
    np.array([-1, 0,1]),
    N_points, N_points)
P_2 = sparse.kronsum(D,D)
T_hat = (-1/2)* P_2 / del_s**2

def potential (x_grid,y_grid):
    return k*((x_grid)**2 + (y_grid)**2)/2

V_hat = sparse.diags(potential(x_grid,y_grid).reshape(N_points*N_points),(0))

H_hat = T_hat + V_hat

E , psi = eigsh(H_hat,which='SM')



In [156]:
for i in range(6):
    fig = go.Figure(go.Surface(x=x_grid,y=y_grid,z=psi.T[i].reshape((N_points,N_points)),
        contours = {
            "z": {"show": True, "start": -1, "end": 1, "size": 0.005}
        },    
        colorscale='Aggrnyl',
        colorbar=dict(thickness=20, ticklen=4),
    ))
    fig.update_layout(title = f"No.{i} Wavefunction ",template='plotly_dark',showlegend=True)
    fig.write_html(f"C:\\PROGRAMS\\112_summer_assignment\\result\\2-d_harmonic_oscillator_potential\\py-no{i}state.html", include_plotlyjs='cdn')


In [5]:
def Hermite(n,x):
    if n == 0:
        return 1
    elif n == 1:
        return x
    elif n == 2:
        return 4*x**2-2

In [157]:


def analytic(n,m,x,y):
    x_norm = np.sqrt(M*omega/h_bar)*x
    y_norm = np.sqrt(M*omega/h_bar)*y
    return (2**(n+m)*np.math.factorial(n)*np.math.factorial(m)*np.pi)**(-1/2)*np.exp((-x_norm**2-y_norm**2)/2)*Hermite(m,x_norm)*Hermite(n,y_norm)

def rotate(x_grid, y_grid, RotRad=0):
    RotMatrix = np.array([[np.cos(RotRad),  np.sin(RotRad)],
                          [-np.sin(RotRad), np.cos(RotRad)]])
    return np.einsum('ji, mni -> jmn', RotMatrix, np.dstack([x_grid, y_grid]))

In [159]:
quantized_num = ((0,0),(1,0),(0,1),(2,0),(1,1),(1,1))
serial_prefix = ('th','st','nd','rd','th','th')

for i in range(6):
    numer = psi.T[i].reshape((N_points,N_points))
    norms = np.sqrt(np.sum(numer**2))
    numer /= norms

    fig = make_subplots(rows=1, cols=2
    ,specs=[[{"type": "scene"}, {"type": "xy"}]],subplot_titles=("Analytic vs Numercial", "Absolute error"))

    MSE = []
    for step in np.arange(0, 360, 3):
        rotated_grid =  rotate(x_grid,y_grid, step/180*np.pi)
        x_rotate = rotated_grid[0]
        y_rotate = rotated_grid[1]
        m,n = quantized_num[i]
        analy = analytic(m,n,x_rotate,y_rotate)
        norms = np.sqrt(np.sum(analy**2))
        analy /= norms
        
        fig.add_trace(go.Surface(name="analytic solution",x=x_grid,y=y_grid,z=analy, visible=False, colorbar=dict(title="Analytic",thickness = 10, x=-0.2)),row=1, col=1)
        fig.add_trace(go.Heatmap(name="absolute error",z=np.abs(analy-numer),visible=False, colorscale='RdBu',colorbar=dict(title="Error",thickness = 10)),row=1, col=2)
        MSE.append(((analy-numer)**2).mean())

    fig.data[0].visible = True
    fig.data[1].visible = True

    ### INVERSE 3RD STATE ###
    if i== 3:
         numer *= -1

    fig.add_trace(go.Surface(name="numercial solution",x=x_grid,y=y_grid,z=numer,colorscale='Aggrnyl',colorbar=dict(title="Numercial",thickness = 10, x=-0.1)))

    steps = []
    for step_index in range((len(fig.data)-1)//2):
        step = dict(
            label = step_index*3,
            method="update",
            args=[{"visible": [False] * (len(fig.data)-1) +  [True]},
            {"title":f" {i}{serial_prefix[i]} eigenvecter [m,n = {m},{n}] -MSE: {MSE[step_index]:.4E}"}
            ],  # layout attribute
        )
        
        step["args"][0]["visible"][step_index*2] = True  
        step["args"][0]["visible"][step_index*2+1] = True
        
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Rotated angle: "},
        pad={"t": 50},
        steps=steps
    )]


    fig.update_layout(template='plotly_dark')
    fig.update_layout(sliders=sliders)

    fig.write_html(f"C:\\PROGRAMS\\112_summer_assignment\\result\\2-D_harmonic_oscillator_potential_with_analysis_sol\\{i}{serial_prefix[i]}_eigenvec.html", include_plotlyjs='cdn')


In [160]:
E

array([31.59724838, 63.16892712, 63.16892712, 94.68940414, 94.68940414,
       94.74060587])